<a href="https://colab.research.google.com/github/mentecuantica/15-minute-apps/blob/master/3d_task_NLP_and_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install -q -U spacy

ERROR: Operation cancelled by user


In [12]:
!python -m spacy download ru_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-28 02:39:52.465041: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [33]:
import re
import pandas as pd
import numpy as np
import gensim
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import spacy
from gensim import corpora, models
from pymorphy2 import MorphAnalyzer

In [14]:
with open('/content/perazki1.txt','r') as f:
  l=f.readlines()

In [15]:
with open('/content/stopwords.txt','r') as f:
  stop=sum([re.split('\W',x) for x in f.readlines()],[])

In [36]:
poems_count = len([ l[i-5:i-1] for i,x in enumerate(l) if i%5==0])
print(f"Poems count total: {poems_count}")

Poems count total: 38071


In [34]:
nlp=spacy.load('ru_core_news_sm')

In [43]:
poems = []
for i,x in enumerate(l):
  if i%5==0:
    poem = ''.join(l[i-5:i-1])
    poems.append(poem)


poems[0]


'свинья везде найдет работу\nответил мне гиппопатам\nи вздор задумчивый направил\nкудато в сторону болот\n'

In [45]:
[ w.text for w in nlp(poems[0]).doc if w.pos_ in ['ADJ','NOUN','VERB']]

#adj_noun_verb
#poems=[ ''.join(l[i-5:i-1]) for i,x in enumerate(l) if i%5==0]

['свинья',
 'найдет',
 'работу',
 'ответил',
 'гиппопатам',
 'задумчивый',
 'направил',
 'кудато',
 'сторону',
 'болот']

In [50]:
nlp(poems[1]).doc

а где тут руль спросил гагарин
деревня буркнул королёв
ещё спроси а где тут вожжи
ещё поехали скажи

In [51]:
def preproc(x):
  #nlp=spacy.load('ru_core_news_sm')
  return ' '.join([ w.text for w in nlp(x).doc if w.pos_ in ['ADJ','NOUN','VERB']])

In [52]:
preproc(poems[0])

'свинья найдет работу ответил гиппопатам задумчивый направил кудато сторону болот'

In [ ]:
#poems=[ ''.join(l[i-5:i-1]) for i,x in enumerate(l) if i%5==0]

In [53]:
poems1=[preproc(x) for x in poems[0:1000]]

In [54]:
m=TfidfVectorizer(stop_words=stop)#,max_features=100,preprocessor=preproc)

In [55]:
m.fit(poems1)

TfidfVectorizer(stop_words=['6ля', '6лядь', '6лять', 'b3ъeб', 'cock', 'cunt',
                            'e6aль', 'ebal', 'eblan', 'eбaл', 'eбaть', 'eбyч',
                            'eбать', 'eбёт', 'eблантий', 'fuck', 'fucker',
                            'fucking', 'xyёв', 'xyй', 'xyя', 'xуе', '', 'xуй',
                            'xую', 'zaeb', 'zaebal', 'zaebali', 'zaebat',
                            'архипиздрит', ...])

In [58]:
X=pd.DataFrame(m.transform(poems1).toarray(),columns=m.get_feature_names_out())

X.transpose()


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
аборигены,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
абрикосом,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
август,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
авиационных,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
автобус,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ясно,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
яхты,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ящерицы,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ящик,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
poems1


['свинья найдет работу ответил гиппопатам задумчивый направил кудато сторону болот',
 'руль спросил деревня буркнул королёв спроси вожжи поехали скажи',
 'позитивный анатолий послан сумел вернуться бодрым отдохнувшим магнитика привез',
 'оксаной тёмном парке распахивает плащ второй третий ждёт',
 'ольгой овладели раз тоска',
 'смыл говно ребята произнес стало понятно волшебный вечер вернуть',
 'любишь ролевые игры изобразить изобрази человека ушел',
 'жена сбежала собака коты смотрю трудно рыбкам аквариум толкать дверям',
 'искусствоведов группа глядит холст седой строгий сказал',
 'любимый взял локоть подвёл окну показал увижу вовеки заткнусь',
 'покосившемся заборе семен напишет слово жук детям почитать взрослым пища ума',
 'давайте веселиться берём топор садимся круг игра простая бутылку называется топор',
 'полушарий оксаны привлекать самцов управляют остальными максимизируя поток',
 'пустую старую квартиру письмо приносит почтальон дед забери продленки увези',
 'лопатки рудименты 

In [ ]:
# end of cleaning the text

In [ ]:
# start task 2 get top 15 topics for poems

In [62]:
pca=PCA(n_components=15)
pca.fit(X)

C=pd.DataFrame(pca.components_,columns=m.get_feature_names_out())
C

,аборигены,абрикосом,август,авиационных,автобус,автогеном,автомат,автомобиль,автопилот,автосварщику,...,японец,японка,японку,японцев,яранги,ясно,яхты,ящерицы,ящик,ёбнется
0,-0.001296,-0.001149,-0.001680,-0.001132,-0.000757,-0.002017,-0.001338,-0.001338,0.000196,-0.001453,...,-0.002408,-0.002408,-0.002408,0.000154,-0.001350,-0.000987,-0.003119,-0.001887,-0.001065,-0.001586
1,-0.000729,-0.001349,-0.001456,-0.001390,-0.000582,-0.001937,-0.001667,-0.001667,-0.001292,0.000210,...,-0.001827,-0.001827,-0.001827,-0.001520,-0.001520,-0.000610,0.006093,-0.002207,0.000551,0.000610
2,0.002102,-0.000359,-0.000513,0.003110,0.001905,-0.000057,0.000043,0.000043,-0.000474,-0.000918,...,-0.000454,-0.000454,-0.000454,-0.000613,0.000152,0.002610,-0.004086,-0.000502,-0.003946,0.010963
3,-0.001441,0.000358,0.000196,-0.001457,-0.004249,-0.008024,0.003222,0.003222,0.000899,-0.004705,...,0.005892,0.005892,0.005892,0.002468,0.000433,-0.000686,0.002254,0.000902,-0.001530,-0.018043
4,-0.002007,-0.000424,0.000476,-0.000583,-0.000801,0.031871,0.001818,0.001818,-0.000341,-0.001408,...,0.003287,0.003287,0.003287,0.000607,-0.000784,-0.000816,0.009009,-0.000490,-0.006503,-0.017723
5,-0.000537,-0.000183,0.001734,-0.000752,-0.001780,-0.004864,0.000982,0.000982,-0.000051,0.003347,...,0.009358,0.009358,0.009358,0.001549,0.000105,-0.000352,0.009062,-0.000638,-0.003908,-0.001705
6,0.001493,0.000059,-0.001586,-0.003707,0.002350,-0.005347,-0.001465,-0.001465,-0.000349,0.004763,...,0.003212,0.003212,0.003212,0.000572,-0.000095,-0.003100,-0.000522,-0.000608,-0.000714,0.004466
7,-0.001460,-0.001614,-0.000110,-0.004802,-0.002373,-0.001759,-0.002442,-0.002442,-0.003068,-0.012308,...,0.002857,0.002857,0.002857,-0.000983,-0.001885,0.000454,0.015203,-0.003506,-0.011013,0.005506
8,-0.001582,-0.001327,-0.001603,-0.001552,-0.002379,-0.007857,-0.001970,-0.001970,-0.004119,0.003546,...,-0.008619,-0.008619,-0.008619,-0.003530,-0.002536,-0.004242,-0.007071,-0.002437,-0.006062,0.007459
9,0.002251,-0.000446,0.000621,-0.003181,-0.004270,-0.001300,-0.000384,-0.000384,-0.001277,0.015481,...,-0.005519,-0.005519,-0.005519,0.004044,-0.000547,-0.000907,0.008906,-0.001445,-0.008282,0.009423


In [ ]:
[list(row[(row>0.1)|(row<-0.1)].index) for i,row in C.iterrows()]

In [ ]:
text_poems = '\n'.join(poems1)
len(text_poems)
#print(text)

In [ ]:
import re
import keras
from sklearn.model_selection import train_test_split

tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])
one_hot_results = tokenizer.texts_to_matrix(text, mode='binary')
#print(len(sequences))
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))



In [ ]:
vocab_size = len(tokenizer.word_index) + 1
#vocab_size = 5828
print("Vocabulary size: ", vocab_size)
sequences = keras.utils.pad_sequences(sequences, padding='pre', maxlen=50)
x_train, x_test, y_train, y_test = train_test_split(sequences, one_hot_results, test_size=1, random_state=42)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)
#print(tokenizer.word_index)


In [ ]:
import keras
from keras.layers import Input, LSTM, Dense, concatenate, Reshape
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences 

import tensorflow as tf

In [ ]:
def get_poem_data(text_poems):
    text_poems = text_poems.lower() # Приведение к нижнему регистру
    text_poems = text_poems.replace('\n', ' ') # Замена переносов строк на пробелы
    text_poems = text_poems.replace('[^\w\s]', '') # Удаление знаков препинания
    # Разбиение текста стихов на отдельные стихи
    poems = text_poems.split('\n\n')

    # Создание словаря всех уникальных слов
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(poems)
    word_index = tokenizer.word_index

    # Преобразование стихов в последовательности слов
    sequences = []
    for poem in poems:
        poem_seq = tokenizer.texts_to_sequences([poem])[0]
        sequences.append(poem_seq)

    # Создание текста, слов и labels для модели
    text = []
    words = []
    labels = []

    for poem_seq in sequences:
      for i in range(len(poem_seq)-2):
          text.append(poem_seq[:i+1])
          words.append(poem_seq[i+1])
          labels.append(poem_seq[i+2])

    labels = keras.utils.to_categorical(labels, num_classes=len(word_index)+1)
    print(labels.shape)
    max_len =  labels.shape[1] #max([len(t) for t in text])
    print(f"Max len: {max_len}")
    print(f"Word index: {word_index}")
    text = pad_sequences(text, maxlen=max_len, padding='pre')
    #text = keras.utils.to_categorical(text, num_classes=len(word_index)+1)
    # Паддинг текста до одинаковой длины
    
    

    # Преобразование слов и labels в one-hot вектора
    words = keras.utils.to_categorical(words, num_classes=len(word_index)+1)
    text = text.reshape(32, 100, 1)
    

    print(f"text shape {text.shape}, words shape {words.shape}, labels.shape {labels.shape}")
    text = np.expand_dims(text,axis=1)
    words = np.expand_dims(words, axis=-1)
    labels = np.expand_dims(labels, axis=-1)
    return text, words, labels



In [ ]:

# Загрузка данных стихов из файла
text, words, labels = get_poem_data(text_poems)
word_to_generate = input("Enter word: ")
word_to_gen_tensor = tf.convert_to_tensor(word_to_generate)
print(f"Tensor shape :{word_to_gen_tensor.get_shape()}")
text_dim = text.shape[1]
print(words.shape)
print(labels.shape)
#print(text)
##print(words)
#print(labels)
# Вычисление размера словаря (количество уникальных слов в тексте)
#vocab_size = len(set(text_poems))
vocab_size = (words.shape)[1]
print(f"Vocabular: {vocab_size}")
# Получение списка уникальных символов
chars = sorted(list(set(text_poems)))

# Количество уникальных символов
input_dim_text = len(chars)
input_dim = (words.shape)[1]
input_dim_text,input_dim_word = vocab_size, vocab_size
print(f"input_dim_text: {input_dim_text}, word: {input_dim_word}")
# Создаем слой входных данных, который принимает текст стихов и заданное слово
# input_text = Input(shape=(timesteps, input_dim))
# input_word = Input(shape=(timesteps, input_dim))
timesteps = 32

# from keras.models import Model
# from keras.layers import Input, LSTM, Dense, concatenate

def create_model(timesteps, input_dim_text,input_dim_word):
    # Создание входных тензоров для текста и слова
    input_text = Input(shape=[timesteps, input_dim_text])
    print(f"INPUT TEXT TENSOR: {input_text.shape}")
    input_word = Input(shape=[timesteps, input_dim_word])
    print(f"INPUT TEXT TENSOR: {input_word.shape}")
    # Объединение тензоров с помощью слоя concatenate
    inputs = concatenate([input_text, input_word])
    print(f"INPUTS SHAPE : {inputs.shape}")
    # Создание слоя LSTM
    hidden = LSTM(32)(inputs)

    # Добавление полносвязного слоя
    output = Dense(units=input_dim, activation='softmax')(hidden)

    # Создание модели
    model = Model(inputs=[input_text, input_word], outputs=output)

    # Компиляция модели
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model


model =create_model(timesteps, vocab_size, vocab_size)
# Компилируем модель, выбрав оптимизатор, функцию потерь и метрику качества
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


# Преобразование списков в тензоры
#text = tf.convert_to_tensor(text)
#words = tf.convert_to_tensor(words)
#labels = tf.convert_to_tensor(labels)

# Разбиение тензора по измерению 1
#text_parts = tf.unstack(text, axis=1)
#word_parts = tf.unstack(words, axis=1)

# Объединение разбитых частей

#inputs = [tf.concat([t, w], axis=1) for t, w in zip(text_parts, word_parts)]

# Обучаем модель на наборе данных стихов
model.fit([text, words], labels, epochs=10, batch_size=32)

# Используем модель для генерации стихов
#generated_poem = model.predict([text, word])


In [ ]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# Создаем слой входных данных, который принимает текст стихов и заданное слово
input_text = Input(s)
input_word = Input()
word = input("Enter word:")

# Создаем скрытые слои нейросети с использованием LSTM
hidden = LSTM(64)(input_text, input_word)

# Создаем выходной слой с использованием Dense с функцией активации softmax
output = Dense(vocab_size, activation='softmax')(hidden)

# Создаем модель нейросети, объединяя все слои
model = Model([input_text, input_word], output)

# Компилируем модель, выбрав оптимизатор, функцию потерь и метрику качества
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем модель на наборе данных стихов
model.fit([text, word], labels, epochs=10, batch_size=32)

# Используем модель для генерации стихов
generated_poem = model.predict([text, word])


In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 50, input_length=50))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
r = model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(r)
history = model.fit(x_train,y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test),verbose=2)

In [ ]:
# Генерация стихов
seed_text = input("Enter the code:")
next_words = 100

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = keras.utils.pad_sequences([token_list], maxlen=50, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    output_word = ""
    for word, index in word_index.items():
        if index == np.argmax(predicted):
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)




In [ ]:
X0=[timm[i-1:i] for i in range(2,len(timm))]
Y0=[timm[i][0] for i in range(2,len(timm))]

In [ ]:
X1=np.reshape(X0,(len(X0),1,1))
Y1=np.reshape(Y0,(len(Y0),1,1))

In [ ]:
Y1.shape

In [ ]:
X1.shape

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(X1.shape[1], X1.shape[2])))
model.add(Flatten())
model.add(Dense(20))
model.compile(loss='MAE', optimizer='adam')

In [ ]:
model.fit(X1,Y1,epochs=10)

In [ ]:
model.predict(X1).shape

In [ ]:
def get_word(y):
  #result = []
  
  #for item in list(d.items()):
  #  print(item)
  #  if item[1]==y:
  #    result.append(item[0])
  #return result
  return [x[0] for x in list(d.items()) if x[1]==y]

In [ ]:
truth=[get_word(ti[i]) for i in range(2,len(ti))]
#np.rint(scaler.inverse_transform(model.predict(X1)))

In [ ]:
predict_round_integer_model = np.rint(scaler.inverse_transform(model.predict(X1)))
# predict = []
# for word in round_integer_model:
#   predict.append(get_word(word))

#predict=[get_word(x) for x in np.rint(scaler.inverse_transform(model.predict(X1)))]


In [ ]:
import pandas as pd

pd.DataFrame([truth,predict_round_integer_model]).transpose()